# L4: Process Inputs: Chain of Thought Reasoning

Chain of Thought (CoT) reasoning is a prompting technique used in large language models (LLMs) where the model is encouraged to explain its reasoning step-by-step before giving the final answer. 

This mimics human-like logical thinking and improves performance, especially for tasks requiring reasoning, such as math problems, logic puzzles, and multi-step decision-making.

In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## Chain-of-Thought Prompting

In [3]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [4]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is comparing the prices of two specific products.
Step 2:#### Both products are available in the list of products provided.
Step 3:#### The assumption made by the user is that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The TechPro Desktop is priced at $999.99, and the BlueWave Chromebook is priced at $249.99. Therefore, the BlueWave Chromebook is $750 cheaper than the TechPro Desktop.
Response to user:#### The BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop.


In [5]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about a specific product category (TVs) and not a specific product.
Step 2:#### N/A
Step 3:#### N/A
Step 4:#### N/A
Response to user:#### We currently do not sell TVs. Our store specializes in computers and laptops. If you have any questions regarding our available products, feel free to ask!


## Inner Monologue
- Since we asked the LLM to separate its reasoning steps by a delimiter, we can hide the chain-of-thought reasoning from the final output that the user sees.

In [6]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

We currently do not sell TVs. Our store specializes in computers and laptops. If you have any questions regarding our available products, feel free to ask!


In [7]:
delimiter = "####"
system_message = f"""
Follow these steps to answer customer queries from Nepali users.
The customer query will be delimited with four hashtags, i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is asking a question about a specific product or products. 
General product category doesn't count. 

Step 2:{delimiter} If the user is asking about specific products, identify whether 
the products are in the following list.
All available products: 
1. Product: Prensu RedmiNotebook
   Category: Laptops
   Brand: Prensu
   Model Number: HT-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: Lightweight laptop perfect for students and office use.
   Price: Rs. 92,000

2. Product: Kastamandav Gaming Laptop
   Category: Laptops
   Brand: Kastamandav Tech
   Model Number: GK-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: High-performance gaming laptop for serious gamers.
   Price: Rs. 152,000

3. Product: Sneha Convertible
   Category: Laptops
   Brand: Sneha
   Model Number: SG-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: Flexible convertible laptop for creative professionals.
   Price: Rs. 84,000

4. Product: Sachhyam Desktop
   Category: Desktop Computers
   Brand: Sachhyam
   Model Number: HT-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: Powerful desktop for work and multimedia.
   Price: Rs. 110,000

5. Product: Ascol Chromebook
   Category: Budget Laptops
   Brand: Ascol
   Model Number: JT-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: Simple and affordable laptop for school students.
   Price: Rs. 28,000

Step 3:{delimiter} If the message contains products in the list above, list any assumptions that the 
user is making in their message, e.g. that Laptop X is faster than Laptop Y, or that a product has longer warranty.

Step 4:{delimiter} If the user made any assumptions, verify whether the assumption is true based on the product info.

Step 5:{delimiter} First, politely correct the customer's incorrect assumptions if applicable. 
Only mention or reference products in the list above, as these are the only products sold. 
Respond to the customer in a friendly tone using their informal Nepali style (in English letters).

Use the following format:
Step 1:{delimiter} <your step 1 reasoning>
Step 2:{delimiter} <your step 2 reasoning>
Step 3:{delimiter} <your step 3 reasoning>
Step 4:{delimiter} <your step 4 reasoning>
Response to user:{delimiter} <your response in informal Nepali-English>
"""


In [8]:
user_message = "Maile Sneha Convertible kinya thye, tara 2 barsha warranty ho bhanera suneko thye, ho ra?"
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about a specific product, Sneha Convertible.
Step 2:#### The product "Sneha Convertible" is in the list of available products.
Step 3:#### The user is assuming that the warranty for the Sneha Convertible is 2 years.
Step 4:#### The warranty for the Sneha Convertible is actually 1 year, not 2 years.

Response to user:#### Oh, sorry for the confusion, dai. Sneha Convertible ko warranty ek barsa matra ho ra, dui barsa haina. Tara laptop ramro chha, flexible ra creative professionals ko lagi perfect chha.


In [12]:
response = get_completion_from_messages(messages)

# Split and extract only the part after 'Response to user:####'
delimiter = "####"
final_response = response.split(f"Response to user:{delimiter}")[-1].strip()
print(final_response)


Oh, sorry for the confusion, dai. Sneha Convertible ko warranty ek barsa matra ho ra, dui barsa haina. Tara laptop ramro chha, flexible ra creative professionals ko lagi perfect chha.


In [14]:
delimiter = "####"
user_message = "Do you sell TVs?"

# Format messages properly
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_message}{delimiter}"}
]

# Call the model
response = get_completion_from_messages(messages)

# Extract only the final response
final_response = response.split(f"Response to user:{delimiter}")[-1].strip()
print(final_response)


Sorry, we currently do not sell TVs. If you are looking for laptops or desktop computers, feel free to ask for more information.


In [15]:
user_message = "Ascol Chromebook le Photoshop support garcha ki nai?"
delimiter = "####"
# Format messages properly
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_message}{delimiter}"}
]

# Call the model
response = get_completion_from_messages(messages)

# Extract only the final response
final_response = response.split(f"Response to user:{delimiter}")[-1].strip()
print(final_response)

Hey there! The Ascol Chromebook might struggle to run Photoshop smoothly due to its limited RAM and storage capacity. It's more suitable for basic tasks like browsing and document editing.


In [ ]:
user_message = "Yo Sachhyam Desktop ko price ali mahango jasto cha, kina yesto costly cha?"
delimiter = "####"
# Format messages properly
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_message}{delimiter}"}
]

# Call the model
response = get_completion_from_messages(messages)

# Extract only the final response
final_response = response.split(f"Response to user:{delimiter}")[-1].strip()
print(final_response)